In [1]:
from pymorphit.pymorphit import Morphit
import re
import itertools
import random as rd

## Obiettivo: generare un set di frasi del tipo "dove si trovano le mie bollette"
### Scrivo delle liste-vocabolario immaginando possibili alternative lessicali

In [2]:
mia = ["mia", ""]
bolletta = ["bolletta", "bollette"]
essere = ["sono", "stare", "si* trova"] # l'asterisco permette di segnalare a pymorphit che la parola precedente è invariante

### Posso interrogare wordnet italiano se non mi vengono in mente sinonimi

In [3]:
def synonyms(lemma):
    from nltk.corpus import wordnet as wn
    synmorph = {}
    wnet_lemmas = wn.lemmas(lemma, lang="ita")
    for index, wnet_lemma in enumerate(wnet_lemmas):
        syns = wnet_lemma.synset()
        m = re.findall("\'([^']*)\'", str(syns))
        synmorph.update({m[0]: syns.lemma_names(lang="ita")})
    return synmorph


synonyms("bolletta")

{'bill.n.02': ['bolletta', 'bulletta', 'conto', 'fattura'],
 'receipt.n.02': ['bolletta',
  'bulletta',
  'contromarca',
  'polizza',
  'quetazione',
  'quietanza',
  'quietazione',
  'ricevuta',
  'ricevuta_fiscale',
  'scontrino',
  'ticket']}

### "fattura" per esempio è un buon sinonimo di 'bolletta', lo aggiungo alla mia variabile

In [4]:
bolletta.append("fattura")

## Le mie liste-vocabolario sono un po' piccole, ma vediamo quante combinazioni posso generare a partire da queste. Tante sono le combinazioni, tante le frasi uniche che posso generare!
### Scrivo una semplice funzione con itertools, che prende in argomento le mie variabili (nell'ordine naturale della frase)

In [5]:
def combo(*args):
    slots = [i for i in args]
    return [combo for combo in itertools.product(*slots)]

combos1 = combo(essere, mia, bolletta)
print(f'Le possibili combinazioni sono in tutto {len(combos1)}')
print()
combos1

Le possibili combinazioni sono in tutto 18



[('sono', 'mia', 'bolletta'),
 ('sono', 'mia', 'bollette'),
 ('sono', 'mia', 'fattura'),
 ('sono', '', 'bolletta'),
 ('sono', '', 'bollette'),
 ('sono', '', 'fattura'),
 ('stare', 'mia', 'bolletta'),
 ('stare', 'mia', 'bollette'),
 ('stare', 'mia', 'fattura'),
 ('stare', '', 'bolletta'),
 ('stare', '', 'bollette'),
 ('stare', '', 'fattura'),
 ('si* trova', 'mia', 'bolletta'),
 ('si* trova', 'mia', 'bollette'),
 ('si* trova', 'mia', 'fattura'),
 ('si* trova', '', 'bolletta'),
 ('si* trova', '', 'bollette'),
 ('si* trova', '', 'fattura')]

### La lista 'bolletta' contiene le parole destinate a essere la testa della frase, ossia la parola alla quale tutte le altre si devono accordare per generare una frase morfologicamente corretta
### Qui entra in gioco Pymorphit: per ogni combinazione viene inizializzato l'oggetto Morphit corrispondente alla testa. Per ragioni di efficienza, le parole già viste da Morphit vengono stoccate (funzione store)

In [6]:
seen = {}

def store(w, seen=seen):
    if w not in seen.keys():
        seen[w] = Morphit(w, "NOUN")
    return seen[w]

## Adesso ciclo sulla lista di combinazioni e genero le frasi, tutte correttamente concordate con i metodi di Pymorphit

In [7]:
gen_sents = []

for essere, mia, bill in combos1:
    
    # h sta per head: la parola a cui si accordano le altri parti del discorso
    h = store(bill)
    
    # i verbi di tipo "essere" di accordano al soggetto "bolletta" e cosivvia. 
    gen_sents.append(f'dove {h.agr(essere)} {h.article()} {h.agr(mia)} {h.word}?')

In [8]:
gen_sents

['dove é la mia bolletta?',
 'dove sono le mie bollette?',
 'dove é la mia fattura?',
 'dove é la  bolletta?',
 'dove sono le  bollette?',
 'dove é la  fattura?',
 'dove sta la mia bolletta?',
 'dove stanno le mie bollette?',
 'dove sta la mia fattura?',
 'dove sta la  bolletta?',
 'dove stanno le  bollette?',
 'dove sta la  fattura?',
 'dove si trova la mia bolletta?',
 'dove si trovano le mie bollette?',
 'dove si trova la mia fattura?',
 'dove si trova la  bolletta?',
 'dove si trovano le  bollette?',
 'dove si trova la  fattura?']

### Ci sono un po' di doppi spazi. Puliamo le frasi con una funzione di formattazione

In [9]:
def formatt(frase):
    punkt1 = re.sub(r'([A-z])( )([,.?!"])',r"\1\3", frase)
    punkt2 = re.sub(r'(\')( )([A-z])',r"\1\3", punkt1)
    formatted = re.sub(r"(^|[.?!])\s*([a-zA-Zè])", lambda p: p.group(0).upper(), punkt2)
    return re.sub(" +"," ", formatted)

[formatt(i) for i in gen_sents]

['Dove é la mia bolletta?',
 'Dove sono le mie bollette?',
 'Dove é la mia fattura?',
 'Dove é la bolletta?',
 'Dove sono le bollette?',
 'Dove é la fattura?',
 'Dove sta la mia bolletta?',
 'Dove stanno le mie bollette?',
 'Dove sta la mia fattura?',
 'Dove sta la bolletta?',
 'Dove stanno le bollette?',
 'Dove sta la fattura?',
 'Dove si trova la mia bolletta?',
 'Dove si trovano le mie bollette?',
 'Dove si trova la mia fattura?',
 'Dove si trova la bolletta?',
 'Dove si trovano le bollette?',
 'Dove si trova la fattura?']

## Adesso qualcosa di più elaborato...

In [10]:
non = ["dove","non"]
trovo = ["trovo", "vedo", "posso vedere"]
mia = ["mia", ""]
bolletta = ["bolletta", "bollette", "fatture"]
su = ["in", "su"]
area = ["area", "spazio"]
clienti = ['riservato', 'clienti*', "personale"]


combos2 = combo(non, trovo, mia, bolletta, su, area, clienti)
print(f'Le possibili combinazioni sono in tutto {len(combos2)}')

Le possibili combinazioni sono in tutto 432


In [11]:
gen_sents = []

for non, trovo, mia, bolletta, su, area, clienti in combos2:
    
    # qui abbiamo due teste, perché il secondo sintagma si accorda alla variabile "area"
    h1 = store(bolletta)
    h2 = store(area)
    
    gen_sents.append(f'{non} {trovo} {h1.article()} {h1.agr(mia)} {h1.word} {h2.preposition(su)} {h2.word} {h2.agr(clienti)}')

In [12]:
# formatto

gen_sents_format = [formatt(i) for i in gen_sents]

### Anche se sono tutte diverse, la ripetitività di queste frasi è ovviamente molto alta, perché il vocabolario di riferimento è piccolo. Potrei volerne prendere solo un subset di 100 frasi

In [13]:
rd.sample(gen_sents_format, 100)

['Non vedo le bollette sullo spazio riservato',
 "Dove trovo le bollette nell'area clienti",
 'Non trovo le bollette nello spazio riservato',
 "Dove trovo le mie bollette sull'area riservata",
 "Non posso vedere le mie bollette nell'area clienti",
 'Non trovo le mie fatture nello spazio personale',
 'Non trovo la mia bolletta sullo spazio personale',
 "Non vedo la mia bolletta sull'area personale",
 "Dove vedo le mie bollette sull'area riservata",
 "Dove posso vedere le mie bollette nell'area riservata",
 "Non vedo le mie fatture nell'area clienti",
 "Dove trovo la mia bolletta sull'area riservata",
 'Non posso vedere le fatture sullo spazio personale',
 'Non posso vedere la bolletta nello spazio personale',
 "Dove vedo le bollette sull'area personale",
 'Dove vedo le mie fatture nello spazio personale',
 'Dove posso vedere la mia bolletta sullo spazio personale',
 "Dove vedo le mie fatture sull'area riservata",
 "Dove posso vedere le mie bollette nell'area personale",
 'Dove trovo le 